In [ ]:
!pip install tqdm pandas scikit-learn openpyxl 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from dataclasses import make_dataclass
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
INPUT_DATA = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/retete_output_scores.xlsx"
OUTPUT_DATA_ML = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/output_data.xlsx"
OUTPUT_DATA_ML_TEST = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/output_data_test.xlsx"
OUTPUT_DATA_ML_PREDICTED = "/content/drive/MyDrive/Colab Notebooks/hyperion/ml/output_data_predicted.xlsx"

In [ ]:
df = pd.read_excel(OUTPUT_DATA_ML, sheet_name="Sheet_1")
df_validity = df.head(100)
test_df = pd.read_excel(OUTPUT_DATA_ML_TEST,sheet_name="Sheet_1")

In [ ]:
df_original = pd.read_excel(INPUT_DATA, sheet_name="retete cardiovascuar")

In [ ]:
X=[]
y=[]
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    feature=[int(row["diag"]),   int(row["sex"]),  int(row["age"]),    int(row["diag_type"]),int(row["s1"]),int(row['s2'])]
    y.append(int(row["bpfp"]))                       
    X.append(feature)

100%|██████████| 5618/5618 [00:00<00:00, 18091.44it/s]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
clf = RandomForestClassifier(n_estimators=10, max_depth=None,
     min_samples_split=2, random_state=0)
clf.fit(X, y)

RandomForestClassifier(n_estimators=10, random_state=0)

In [ ]:
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    feature=[[int(row["diag"]),   int(row["sex"]),  int(row["age"]),    int(row["diag_type"]),int(row["s1"]),int(row['s2'])]]
    res = clf.predict(feature)
    r_number = int(row['nr'])
    for i2,r2 in df_original.iterrows():
        nr = int(r2['nr rp'])
        if r_number == nr:
            df_original.at[i2,"predicted"]=res

100%|██████████| 675/675 [00:37<00:00, 17.89it/s]


In [ ]:
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()
df_original

In [ ]:
with pd.ExcelWriter(OUTPUT_DATA_ML_PREDICTED) as writer:
    df_original.to_excel(writer, sheet_name="retete cardiovascuar")


# Validation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
counter = 0
for index, row in tqdm(df_validity.iterrows(), total=df_validity.shape[0]):
    feature=[[int(row["diag"]),   int(row["sex"]),  int(row["age"]),    int(row["diag_type"]),int(row["s1"]),int(row['s2'])]]
    res = int(clf.predict(feature))
    acc_score=accuracy_score(int(row["bpfp"]),res)
    print(f"Accuracy: {acc_score}")
    bpfp=int(row["bpfp"])
    if bpfp == res:
        counter+=1
percentage = counter/100
print(f'Probability:{percentage}')

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: ignored